# Generación y Clasificación de Secuencias Genéticas Sintéticas

Este notebook explica paso a paso cómo se genera un dataset sintético de secuencias genéticas, se entrena un clasificador de riesgo y se utiliza un modelo generativo LSTM para crear nuevas secuencias, todo con Python y librerías de IA.

## 1. Importación de Librerías

Se importan las librerías necesarias para la generación de datos, modelado y clasificación.

In [ ]:
import numpy as np
import random
from faker import Faker
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

fake = Faker()

## 2. Generación de Datos Sintéticos

Se crean funciones para generar secuencias genéticas aleatorias y asignar un nivel de riesgo. Luego, se construye un dataset sintético.

In [ ]:
def generar_secuencia_genetica(longitud=8):
    return [random.randint(1, 20) for _ in range(longitud)]

def riesgo_aleatorio():
    return random.choices([0,1,2], weights=[0.6,0.3,0.1])[0]

# Dataset sintético
X = np.array([generar_secuencia_genetica() for _ in range(800)])
y = np.array([riesgo_aleatorio() for _ in range(800)])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## 3. Entrenamiento del Clasificador RandomForest

Se entrena un clasificador RandomForest para predecir el nivel de riesgo de una secuencia genética.

In [ ]:
clf = RandomForestClassifier(n_estimators=50, random_state=42)
clf.fit(X_train, y_train)

print(f"RandomForest accuracy test: {clf.score(X_test, y_test):.2f}")

## 4. Preparación y Entrenamiento del Modelo LSTM

Se prepara el dataset para el modelo generativo LSTM, que aprenderá a predecir la siguiente base genética en una secuencia.

In [ ]:
vocab_size = 21
max_len = 8
embed_dim = 12

# Preparar secuencias para modelo generativo
sequences = []
next_bases = []
for seq in X:
    for i in range(1, len(seq)):
        sequences.append(seq[:i])
        next_bases.append(seq[i])

sequences = pad_sequences(sequences, maxlen=max_len, padding='pre')
next_bases = to_categorical(next_bases, num_classes=vocab_size)

model = Sequential([
    Embedding(vocab_size, embed_dim, input_length=max_len),
    LSTM(64),
    Dense(vocab_size, activation='softmax')
])
model.compile(loss='categorical_crossentropy', optimizer='adam')

model.fit(sequences, next_bases, epochs=5, batch_size=32, verbose=0)

## 5. Diccionario de Traducción de Bases Genéticas

Se define un diccionario para traducir los números de las bases a nombres científicos o ficticios.

In [ ]:
base_map = {
    1:"Adenina",2:"Citosina",3:"Guanina",4:"Timina",5:"Uracilo",
    6:"Elemento X1",7:"Elemento X2",8:"Elemento X3",9:"Elemento X4",10:"Elemento X5",
    11:"Proteína A",12:"Proteína B",13:"Enzima C",14:"Enzima D",
    15:"Mutación Alta",16:"Mutación Baja",17:"Secuencia Críptica",
    18:"Secuencia Activa",19:"Elemento Regulador",20:"Elemento Silenciador"
}

## 6. Funciones de Generación y Traducción de Secuencias

Se implementan funciones para generar nuevas secuencias con el modelo LSTM y traducirlas a texto legible.

In [ ]:
def generar_secuencia(model, seed_seq, longitud=max_len):
    result = seed_seq.copy()
    for _ in range(longitud - len(seed_seq)):
        padded = pad_sequences([result], maxlen=max_len, padding='pre')
        pred = model.predict(padded, verbose=0)[0]
        next_base = np.random.choice(range(vocab_size), p=pred)
        if next_base == 0:  # padding
            break
        result.append(next_base)
    return result

def secuencia_a_texto(seq):
    return ", ".join(base_map.get(b, "?") for b in seq)

## 7. Clase BioFakerIA

Clase que genera especies ficticias con nombre, hábitat, descripción, genoma sintético y nivel de riesgo.

In [ ]:
class BioFakerIA:
    def __init__(self):
        self.fake = Faker()

    def species_name(self):
        return f"{self.fake.last_name()} {self.fake.word().capitalize()}"

    def habitat(self):
        return self.fake.city()

    def description(self):
        descs = [
            "Especie endémica con adaptaciones únicas.",
            "Conocida por su rápido crecimiento.",
            "Posee mecanismos defensivos avanzados.",
            "Importante en el ecosistema local.",
            "Presenta una dieta variada y flexible."
        ]
        return random.choice(descs)

    def generate(self):
        nombre = self.species_name()
        habitat = self.habitat()
        descripcion = self.description()

        seed = generar_secuencia_genetica(3)
        secuencia = generar_secuencia(model, seed)
        texto_gen = secuencia_a_texto(secuencia)

        riesgo = clf.predict([secuencia])[0]
        niveles = {0:"Bajo",1:"Medio",2:"Alto"}

        bio = f"🧬 Especie: {nombre}\n" \
              f"🏞️ Hábitat: {habitat}\n" \
              f"📝 Descripción: {descripcion}\n" \
              f"🧬 Genoma sintético: {texto_gen}\n" \
              f"⚠️ Nivel de riesgo para humanos: {niveles[riesgo]}"
        return bio

## 8. Ejemplo de Generación de Especie Sintética

Se crea una instancia de la clase y se genera una especie ficticia.

In [ ]:
biofaker = BioFakerIA()
print(biofaker.generate())

## 9. Interfaz Web con Streamlit

Puedes ejecutar la siguiente app para generar especies ficticias desde una interfaz web:

```bash
streamlit run biofaker_app.py
```